<a href="https://colab.research.google.com/github/turing-club/info/blob/master/BipedalWalker_COGS_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bipedal Walker - reinforcement learning

## Introduction
We've just shown you how reinforcement learning can be used to let a computer learn a psychology task. Now, we're going to help a robot learn how to walk! 

The process by which the robot will learn the task is based on the exact same principles as the process by which a computer learns to do a go/no-go task. 

Don't be intimidated by all of the code below; we'll take you through the most important parts!

Source: https://github.com/mayurmadnani/BipedalWalker

## Installation
We'll be using OpenAI Gym, a framework that provides ton of games for the AIs we create to learn and play. You can see all of the available environments here: https://gym.openai.com/envs/#classic_control

The environment we will use is BipedalWalker-v2. This is a simulation of a robot that will try to learn to walk on two legs and reach the other side of the screen - all by itself!

Let's phrase the robot's challenge in the terms we've been using so far:
The robot's **stimulus/environment** is a **grassy field**.
The robot's **decision** will be how to align its legs to stand up and walk.
Depending on the robot's decision, the **outcome** will either be a reward (a few points for walking forward) or a punishment (-100 points for falling down).

The robot knows:
- How fast it's going
- How its feet touch the ground
- How far away the ground is from its head (using a sensor of the kind found in self-driving cars)
- Whether it's been rewarded or punished for its actions

Let's install some software that will let us render the robot's little world into a video.

In [0]:
!pip install gym==0.12.1
!apt-get update
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!apt-get install xvfb
!pip install pyvirtualdisplay
!pip -q install pyglet
!pip -q install pyopengl

Now, let's install the physics engine that defines the world in which our robot lives.

In [0]:
!apt-get install swig x11-utils
!pip install box2d box2d-kengz
!pip install pybullet

"Hello world!" Let's set up our robot's world.

In [0]:
# Start virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

# Augmented Random Search

**Augmented random search** is a policy that the robot will use to explore the world (and learn to walk). It will start out by trying out random movements, watch the rewards and punishments it gets for each action, and using the information it has about the world to improve its performance.

In [0]:
import os
import numpy as np
import gym
from gym import wrappers

**Hyperparameters** are variables that define how fast our robot's brain will work. These parameters require a lot of tuning in order to get a good model that will enable the robot to learn efficently.

In [0]:
class HP():
    # Hyperparameters
    def __init__(self,
                 nb_steps=1000,
                 episode_length=2000,
                 learning_rate=0.02,
                 num_deltas=16,
                 num_best_deltas=16,
                 noise=0.03,
                 seed=1,
                 env_name='BipedalWalker-v2',
                 record_every=50):

        self.nb_steps = nb_steps
        self.episode_length = episode_length
        self.learning_rate = learning_rate
        self.num_deltas = num_deltas
        self.num_best_deltas = num_best_deltas
        assert self.num_best_deltas <= self.num_deltas
        self.noise = noise
        self.seed = seed
        self.env_name = env_name
        self.record_every = record_every

Have you taken PSYC 218? If so, this next block of code will make a lot of sense to you. We want to normalize all of the robot's behaviours (the size of its step, speed of walking, etc.) so that we can directly compare parameters with different units. To do this, we'll convert the robot's movement parameters to z-scores!

It mostly happens in the following line:
```
return (inputs - obs_mean) / obs_std
```
This is (current position - mean position) / standard deviation, which is the formula for calculating a z-score!

In [0]:
class Normalizer():
    # Normalizes the inputs
    def __init__(self, nb_inputs):
        self.n = np.zeros(nb_inputs)
        self.mean = np.zeros(nb_inputs)
        self.mean_diff = np.zeros(nb_inputs)
        self.var = np.zeros(nb_inputs)

    def observe(self, x):
        self.n += 1.0
        last_mean = self.mean.copy()
        self.mean += (x - self.mean) / self.n
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff / self.n).clip(min = 1e-2)

    def normalize(self, inputs):
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        return (inputs - obs_mean) / obs_std

Here, we evaluate how well our policy (the robot's program of action) is working, and update its behavioural model accordingly.

Focus on this line:


```
self.theta += self.hp.learning_rate / (self.hp.num_best_deltas * sigma_rewards) * step
```
**self.theta** is the weight assigned to a particular behaviour. From a neuroscience perspective, you can think of it as a long-term potentiation or inhibition (strengthening or weakening) of a neural connection. If the behaviour is beneficial, then you want to strengthen the neurons driving that behaviour (and vice versa).

**self.hp.learning_rate** is the speed at which the model learns. If the model learns too fast, it may overshoot the optimal setup for making a good decision (i.e. making too large of a step and falling over). 

**self.hp.num_best_deltas** represents the robot's past experiences. Here, the robot is 'thinking' about the decisions it made in the past, and the optimal deltas - the optimal shifts in strategy it has made.

**sigma_rewards** is the sum of the rewards the robot has gained for making specific behaviours of that type before. If the action resulted in the robot making progress towards the goal in the past, it's assigned a positive reward. If the action resulted in the robot falling, it's assigned a punishment (negative reward).


In [0]:
class Policy():
    def __init__(self, input_size, output_size, hp):
        self.theta = np.zeros((output_size, input_size))
        self.hp = hp

    def evaluate(self, input, delta = None, direction = None):
        if direction is None:
            return self.theta.dot(input)
        elif direction == "+":
            return (self.theta + self.hp.noise * delta).dot(input)
        elif direction == "-":
            return (self.theta - self.hp.noise * delta).dot(input)

    def sample_deltas(self):
        return [np.random.randn(*self.theta.shape) for _ in range(self.hp.num_deltas)]

    def update(self, rollouts, sigma_rewards):
        # sigma_rewards is the standard deviation of the rewards
        step = np.zeros(self.theta.shape)
        for r_pos, r_neg, delta in rollouts:
            step += (r_pos - r_neg) * delta
        self.theta += self.hp.learning_rate / (self.hp.num_best_deltas * sigma_rewards) * step

Now, it's time to define how our robot is going to learn. We're putting together the world, normalizer, and policy that we defined in the previous cells. 

In **explore**, we're telling the robot to go forth into this brave new world, try out various strategies according to its policies, and note down the rewards and punishments it gets. 

In **train**, it uses the rewards and punishments that it's faced in **explore** to update its behaviour for the next strategy that it will try.

In [0]:
class ARSTrainer():
    def __init__(self,
                 hp=None,
                 input_size=None,
                 output_size=None,
                 normalizer=None,
                 policy=None,
                 monitor_dir=None):

        self.hp = hp or HP()
        np.random.seed(self.hp.seed)
        self.env = gym.make(self.hp.env_name)
        if monitor_dir is not None:
            should_record = lambda i: self.record_video
            self.env = wrappers.Monitor(self.env, monitor_dir, video_callable=should_record, force=True)
        self.hp.episode_length = self.env.spec.timestep_limit or self.hp.episode_length
        self.input_size = input_size or self.env.observation_space.shape[0]
        self.output_size = output_size or self.env.action_space.shape[0]
        self.normalizer = normalizer or Normalizer(self.input_size)
        self.policy = policy or Policy(self.input_size, self.output_size, self.hp)
        self.record_video = False

    # Explore the policy on one specific direction and over one episode
    def explore(self, direction=None, delta=None):
        state = self.env.reset()
        done = False
        num_plays = 0.0
        sum_rewards = 0.0
        while not done and num_plays < self.hp.episode_length:
            self.normalizer.observe(state)
            state = self.normalizer.normalize(state)
            action = self.policy.evaluate(state, delta, direction)
            state, reward, done, _ = self.env.step(action)
            reward = max(min(reward, 1), -1)
            sum_rewards += reward
            num_plays += 1
        return sum_rewards

    def train(self):
        for step in range(self.hp.nb_steps):
            # initialize the random noise deltas and the positive/negative rewards
            deltas = self.policy.sample_deltas()
            positive_rewards = [0] * self.hp.num_deltas
            negative_rewards = [0] * self.hp.num_deltas

            # play an episode each with positive deltas and negative deltas, collect rewards
            for k in range(self.hp.num_deltas):
                positive_rewards[k] = self.explore(direction="+", delta=deltas[k])
                negative_rewards[k] = self.explore(direction="-", delta=deltas[k])
                
            # Compute the standard deviation of all rewards
            sigma_rewards = np.array(positive_rewards + negative_rewards).std()

            # Sort the rollouts by the max(r_pos, r_neg) and select the deltas with best rewards
            scores = {k:max(r_pos, r_neg) for k,(r_pos,r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
            order = sorted(scores.keys(), key = lambda x:scores[x], reverse = True)[:self.hp.num_best_deltas]
            rollouts = [(positive_rewards[k], negative_rewards[k], deltas[k]) for k in order]

            # Update the policy
            self.policy.update(rollouts, sigma_rewards)

            # Only record video during evaluation, every n steps
            if step % self.hp.record_every == 0:
                self.record_video = True
            # Play an episode with the new weights and print the score
            reward_evaluation = self.explore()
            print('Step: ', step, 'Reward: ', reward_evaluation)
            self.record_video = False

This block of code just sets up a directory to which we'll save the videos of the robot's behaviour.

In [0]:
def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

We've defined all of the parameters of the model - now, it's time to unleash the robot upon the little world we've created for it!

This step will begin the robot's training. Please be patient - it takes a while for the robot to be able to walk. In the meantime, you can watch the robot's rewards and punishments. When you feel that you've trained the robot enough, click the play button again to stop the training.

In [0]:
ENV_NAME = 'BipedalWalker-v2'

videos_dir = mkdir('.', 'videos')
monitor_dir = mkdir(videos_dir, ENV_NAME)

hp = HP(env_name=ENV_NAME)
trainer = ARSTrainer(hp=hp, monitor_dir=monitor_dir)
trainer.train()

# Download the episodes

Congratulations, the robot has (hopefully) learned to walk a bit! To see how it performed, run the following code blocks to download a video of the robot's behaviour. That's it!

In [0]:
!ls videos/{ENV_NAME}

In [0]:
from google.colab import files
import glob

for file in glob.glob("videos/{}/openaigym.video.*.mp4".format(ENV_NAME)):
  files.download(file)